In [1]:
import psycopg2
import pandas as pd
import pandas.io.sql as sqlio
import config

### SETUP

In [2]:
SAVE_CSV = False
TEST = True

In [3]:
conn = psycopg2.connect(**(config.config()))
cur = conn.cursor()
sql = "SELECT * FROM evaluation"
df = sqlio.read_sql_query(sql, conn)
conn.close()

/Users/aun/Desktop/Folders/Work/Github/resource-based-checklist-generation/database/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
df['time'] = df['time'].astype(int)
filtered_out = [] if TEST else ['EaDiYc', 'amwWOh']
filtered_df = df[~(df['id'].isin(filtered_out))].groupby('id').filter(lambda group: len(group) == 4)

In [5]:
list_tasks = [
              list(filtered_df.groupby('task').get_group('task1')['result']),
              list(filtered_df.groupby('task').get_group('task2')['result']),
              list(filtered_df.groupby('task').get_group('task3')['result']),
]

### Calculate Avg. Time Performing Each Task

In [6]:
nano_to_sec_convertion = 1e9
completed_evaluation = pd.DataFrame(filtered_df[['id', 'task']])
completed_evaluation['avg'] = filtered_df.sort_values(['id', 'time']).groupby('id')['time'].diff(1) / nano_to_sec_convertion
completed_evaluation.groupby('task')['avg'].mean().dropna().reset_index()

,task,avg
0,task1,876.439302
1,task2,234.404584
2,task3,455.467203


In [7]:
survey_time = filtered_df[(filtered_df['task'] == 'booking') | (filtered_df['task'] == 'task3')].sort_values(['id', 'time']).groupby('id')['time'].diff(1) / nano_to_sec_convertion
avg_time_taken = survey_time.dropna().mean()
print(f'Average time taken to finish all the tasks: {int(avg_time_taken // 60)}:{(avg_time_taken % 60).round(2)} min ({avg_time_taken.round(2)} seconds)')

Average time taken to finish all the tasks: 26:6.31 min (1566.31 seconds)


In [8]:
def objs_exist(arr, fields, exist=1):
  return len([a for a in arr if all(str(a[k]).strip() == str(v) for k, v in fields.items())]) == exist

### Task 1

In [9]:
task1_checklist = []
task_no = 0

for item in list_tasks[task_no]:
  task_checklist = {}
  tx_details = item['information'][0]['details']
  task_checklist['information_name'] = item['information'][0]['name'] == 'Purchased Items'
  task_checklist['customer_id_hide'] = objs_exist(tx_details, {'inputDependencyField': 'customer_id'})
  task_checklist['order_name'] = objs_exist(tx_details, {'name': 'Order ID'})
  task_checklist['total_price'] = objs_exist(tx_details, {'inputDependencyField': 'total_price', 'name': 'Total Price'})
  task_checklist['tx_id'] = objs_exist(tx_details, {'inputDependencyField': 'id', 'queryTable': 'transactions', 'queryField': 'id'})
  task_checklist['tx_id_name'] = objs_exist(tx_details, {'inputDependencyField': 'id', 'queryTable': 'transactions', 'queryField': 'id', 'name': 'Transaction ID'})
  task_checklist['item_id'] = objs_exist(tx_details, {'inputDependencyField': 'id', 'queryTable': 'item_list', 'queryField': 'item_id'})
  task_checklist['item_id_name'] = objs_exist(tx_details, {'inputDependencyField': 'id', 'queryTable': 'item_list', 'queryField': 'item_id', 'name': 'Item ID'})

  task_checklist['c_len'] = len(item['components']) == 1
  task_checklist['c_header'] = objs_exist(item['components'], {'componentType': 'HEADER', 'order': 0, 'name': 'Card Details'})
  card_form = item['components'][0]['children']
  task_checklist['c_child_len'] = task_checklist['c_header'] and len(item['components'][0]['children']) == 3
  task_checklist['c_card_no'] = task_checklist['c_header'] and objs_exist(card_form, {'componentType': 'INPUT', 'name': 'Card Number'})
  task_checklist['c_expire'] = task_checklist['c_header'] and objs_exist(card_form, {'componentType': 'DATE', 'name': 'Expire Date'})
  task_checklist['c_code'] = task_checklist['c_header'] and objs_exist(card_form, {'componentType': 'INPUT', 'name': 'Security Code'})
  task_checklist['c_card_no_ode'] = task_checklist['c_header'] and objs_exist(card_form, {'componentType': 'INPUT', 'name': 'Card Number', 'outputDependency': 'CardDetails', 'outputDependencyField': 'card_no'})
  task_checklist['c_expire_ode'] = task_checklist['c_header'] and objs_exist(card_form, {'componentType': 'DATE', 'name': 'Expire Date', 'outputDependency': 'CardDetails', 'outputDependencyField': 'expire'})
  task_checklist['c_code_ode'] = task_checklist['c_header'] and objs_exist(card_form, {'componentType': 'INPUT', 'name': 'Security Code', 'outputDependency': 'CardDetails', 'outputDependencyField': 'code'})
  task_checklist['c_card_no_ide'] = task_checklist['c_header'] and objs_exist(card_form, {'componentType': 'INPUT', 'name': 'Card Number', 'outputDependency': 'CardDetails', 'inputDependencyField': ''})
  task_checklist['c_expire_ide'] = task_checklist['c_header'] and objs_exist(card_form, {'componentType': 'DATE', 'name': 'Expire Date', 'outputDependency': 'CardDetails', 'inputDependencyField': ''})
  task_checklist['c_code_ide'] = task_checklist['c_header'] and objs_exist(card_form, {'componentType': 'INPUT', 'name': 'Security Code', 'outputDependency': 'CardDetails', 'inputDependencyField': ''})

  task_checklist['accuracy'] = list(task_checklist.values()).count(True) / len(task_checklist.values())
  task1_checklist.append(task_checklist)

In [10]:
df_task1_checklist = pd.DataFrame(task1_checklist)
task1_results = pd.concat([df_task1_checklist, pd.DataFrame(df_task1_checklist.mean(numeric_only=True, axis=0)).transpose()]).reset_index().drop(columns='index')
if SAVE_CSV: task1_results.to_csv('task1_results.csv')
task1_results

,information_name,customer_id_hide,order_name,total_price,tx_id,tx_id_name,item_id,item_id_name,c_len,c_header,...,c_card_no,c_expire,c_code,c_card_no_ode,c_expire_ode,c_code_ode,c_card_no_ide,c_expire_ide,c_code_ide,accuracy
0,False,True,True,True,True,True,True,True,True,True,...,True,False,True,True,False,True,True,False,True,0.8
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,1.0
2,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.5,1.0,1.0,0.5,1.0,1.0,0.5,1.0,0.9


### Task 2

In [11]:
task2_checklist = []
task_no = 1

for item in list_tasks[task_no]:
  task_checklist = {}
  tx_details = item['information'][0]['details']
  task_checklist['information_name'] = item['information'][0]['name'] == 'Purchased Items'
  task_checklist['customer_id_hide'] = objs_exist(tx_details, {'inputDependencyField': 'customer_id'})
  task_checklist['order_name'] = objs_exist(tx_details, {'name': 'Order ID'})
  task_checklist['total_price'] = objs_exist(tx_details, {'inputDependencyField': 'total_price', 'name': 'Total Price'})
  task_checklist['tx_id'] = objs_exist(tx_details, {'inputDependencyField': 'id', 'queryTable': 'transactions', 'queryField': 'id'})
  task_checklist['tx_id_name'] = objs_exist(tx_details, {'inputDependencyField': 'id', 'queryTable': 'transactions', 'queryField': 'id', 'name': 'Transaction ID'})
  task_checklist['item_id'] = objs_exist(tx_details, {'inputDependencyField': 'id', 'queryTable': 'item_list', 'queryField': 'item_id'})
  task_checklist['item_id_name'] = objs_exist(tx_details, {'inputDependencyField': 'id', 'queryTable': 'item_list', 'queryField': 'item_id', 'name': 'Item ID'})

  task_checklist['c_len'] = len(item['components']) == 1
  task_checklist['c_header'] = objs_exist(item['components'], {'componentType': 'HEADER', 'order': 0, 'name': 'Card Details'})
  card_form = item['components'][0]['children']
  task_checklist['c_child_len'] = task_checklist['c_header'] and len(item['components'][0]['children']) == 3
  task_checklist['c_card_no'] = task_checklist['c_header'] and objs_exist(card_form, {'componentType': 'INPUT', 'name': 'Card Number'})
  task_checklist['c_expire'] = task_checklist['c_header'] and objs_exist(card_form, {'componentType': 'DATE', 'name': 'Expire Date'})
  task_checklist['c_code'] = task_checklist['c_header'] and objs_exist(card_form, {'componentType': 'INPUT', 'name': 'Security Code'})
  task_checklist['c_card_no_ode'] = task_checklist['c_header'] and objs_exist(card_form, {'componentType': 'INPUT', 'name': 'Card Number', 'outputDependency': 'CardDetails', 'outputDependencyField': 'card_no'})
  task_checklist['c_expire_ode'] = task_checklist['c_header'] and objs_exist(card_form, {'componentType': 'DATE', 'name': 'Expire Date', 'outputDependency': 'CardDetails', 'outputDependencyField': 'expire'})
  task_checklist['c_code_ode'] = task_checklist['c_header'] and objs_exist(card_form, {'componentType': 'INPUT', 'name': 'Security Code', 'outputDependency': 'CardDetails', 'outputDependencyField': 'code'})
  task_checklist['c_card_no_ide'] = task_checklist['c_header'] and objs_exist(card_form, {'componentType': 'INPUT', 'name': 'Card Number', 'outputDependency': 'CardDetails', 'inputDependencyField': ''})
  task_checklist['c_expire_ide'] = task_checklist['c_header'] and objs_exist(card_form, {'componentType': 'DATE', 'name': 'Expire Date', 'outputDependency': 'CardDetails', 'inputDependencyField': ''})
  task_checklist['c_code_ide'] = task_checklist['c_header'] and objs_exist(card_form, {'componentType': 'INPUT', 'name': 'Security Code', 'outputDependency': 'CardDetails', 'inputDependencyField': ''})
  
  task_checklist['accuracy'] = list(task_checklist.values()).count(True) / len(task_checklist.values())
  task2_checklist.append(task_checklist)

In [12]:
df_task2_checklist = pd.DataFrame(task2_checklist)
task2_results = pd.concat([df_task2_checklist, pd.DataFrame(df_task2_checklist.mean(numeric_only=True, axis=0)).transpose()]).reset_index().drop(columns='index')
if SAVE_CSV: task2_results.to_csv('task2_results.csv')
task2_results

,information_name,customer_id_hide,order_name,total_price,tx_id,tx_id_name,item_id,item_id_name,c_len,c_header,...,c_card_no,c_expire,c_code,c_card_no_ode,c_expire_ode,c_code_ode,c_card_no_ide,c_expire_ide,c_code_ide,accuracy
0,True,True,True,True,True,True,True,True,True,True,...,True,False,True,True,False,True,True,False,True,0.850
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,1.000
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.5,1.0,1.0,0.5,1.0,1.0,0.5,1.0,0.925


### Task 3

In [13]:
task3_checklist = []
task_no = 2

for item in list_tasks[task_no]:
  task_checklist = {}
  
  task_checklist['service_hide'] = len([a for a in item['information'][0]['details'] if a['hide']]) == 4
  task_checklist['contract_hide'] = len([a for a in item['information'][1]['details'] if a['hide']]) == 5
  task_checklist['patient_name'] = len([a for a in item['information'][0]['details'] if a['inputDependencyField'] == 'actorid' and a['queryTable'] == 'staff' and a['queryField'] == 'name']) == 1
  task_checklist['patient_name_fname'] = len([a for a in item['information'][0]['details'] if a['inputDependencyField'] == 'actorid' and a['queryTable'] == 'staff' and a['queryField'] == 'name' and a['name'] == "Provider's Name"]) == 1
  task_checklist['patient_surname_fname'] = len([a for a in item['information'][0]['details'] if a['inputDependencyField'] == 'actorid' and a['queryTable'] == 'staff' and a['queryField'] == 'surname']) == 1
  task_checklist['patient_surname_fname'] = len([a for a in item['information'][0]['details'] if a['inputDependencyField'] == 'actorid' and a['queryTable'] == 'staff' and a['queryField'] == 'surname' and a['name'] == "Provider's Surname"]) == 1

  task_checklist['c_len'] = len(item['components']) == 1
  task_checklist['c_header'] = objs_exist(item['components'], {'componentType': 'HEADER', 'name': 'Open Contract'})
  contract_form = item['components'][0]['children'] if item['components'][0] else []
  task_checklist['c_child_len'] = len(contract_form) == 6
  task_checklist['c_idcontract'] = objs_exist(contract_form, {'inputDependencyField': 'idcontract', 'outputDependencyField': 'idcontract', 'hide': True})
  task_checklist['c_reqservid'] = objs_exist(contract_form, {'inputDependencyField': 'reqservid', 'outputDependencyField': 'reqservid', 'hide': True})
  task_checklist['c_providerid'] = objs_exist(contract_form, {'inputDependencyField': 'providerid', 'outputDependencyField': 'providerid', 'hide': True})
  task_checklist['c_time_requested'] = objs_exist(contract_form, {'inputDependencyField': 'time_requested', 'outputDependencyField': 'time_requested', 'hide': True})
  task_checklist['c_time_opened'] = objs_exist(contract_form, {'inputDependencyField': '', 'outputDependencyField': 'time_opened', 'hide': False, 'name': 'Date Opened', 'componentType': 'DATE', 'required': True})
  task_checklist['c_stateid'] = objs_exist(contract_form, {'inputDependencyField': '', 'outputDependencyField': 'stateid', 'hide': False, 'name': 'Opened State', 'componentType': 'CONSTANT', 'required': True, 'validation': '3'})

  task_checklist['accuracy'] = list(task_checklist.values()).count(True) / len(task_checklist.values())
  task3_checklist.append(task_checklist)

In [14]:
df_task3_checklist = pd.DataFrame(task3_checklist)
task3_results = pd.concat([df_task3_checklist, pd.DataFrame(df_task3_checklist.mean(numeric_only=True, axis=0)).transpose()]).reset_index().drop(columns='index')
if SAVE_CSV: task3_results.to_csv('task3_results.csv')
task3_results

,service_hide,contract_hide,patient_name,patient_name_fname,patient_surname_fname,c_len,c_header,c_child_len,c_idcontract,c_reqservid,c_providerid,c_time_requested,c_time_opened,c_stateid,accuracy
0,False,True,True,False,False,True,False,True,True,True,True,True,True,False,0.642857
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,1.000000
2,0.5,1.0,1.0,0.5,0.5,1.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.821429
